In [1]:
import pandas as pd
from frequentist_treatment_nlp import *
import nltk
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk.stem.snowball import SnowballStemmer
import regex
import re

C:\Users\32mor\anaconda3\envs\tf_test\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\32mor\anaconda3\envs\tf_test\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\32mor\anaconda3\envs\tf_test\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
os.chdir('C:/Users/32mor/OneDrive/Documents/Polytechnique/NLP & Natixis/starting_kit/data/data/train')
data_train = pd.read_json(os.listdir()[1])
speeches = pd.DataFrame(data_train['speech'].values.tolist())
speech_cols = pd.DataFrame(index=speeches.index)

In [3]:
speeches = form_speeches_dataset(data_train)

In [4]:
bank = 'ecb'
speeches[speeches[f'{bank.upper()}_speeches'].isnull()]

,ECB_speeches,FED_speeches,Value_number,day_number,Principal_key
2,None,None,0,2,0-2
3,None,None,0,3,0-3
9,None,None,0,9,0-9
10,None,None,0,10,0-10
12,None,None,0,12,0-12
...,...,...,...,...,...
25072,None,None,1253,12,1253-12
25075,None,None,1253,15,1253-15
25076,None,None,1253,16,1253-16
25077,None,None,1253,17,1253-17


In [5]:
X_test = transform_nlp_dta(speeches, 'ecb')



In [11]:
tf_idf_final = tf_idf_pca_reduction(X_test)

In [14]:
tf_idf_final.shape

(25080, 300)

In [16]:
day_may = speeches['day_number'].may() + 1
speeches_max = speeches['Value_number'].max() + 1
components = 300

In [27]:
data_lstm = tf_idf_final.reshape((speeches_max, day_max, components))

In [39]:
from sklearn.model_selection import train_test_split

In [62]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,GRU,LSTM,Embedding, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import SpatialDropout1D,Dropout,Bidirectional,Conv1D,GlobalMaxPooling1D, MaxPooling1D,Flatten, InputLayer, Input
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
import tensorflow as tf

In [23]:
tf.config.list_physical_devices()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [25]:
y_train_cat = data_train['target_classif'].values
y_train_reg = data_train['target_reg']

In [48]:
X_train, X_test, y_train, y_test = train_test_split(data_lstm, data_train[['target_reg','target_classif']], 
                                                    test_size=0.2)

In [49]:
y_train_cat = y_train['target_classif']
y_test_cat = y_test['target_classif']
y_train_reg = y_test['target_reg']
y_test_reg = y_test['target_reg']

In [50]:
visible = Input(shape=(data_lstm.shape[1], data_lstm.shape[2]))
hidden1 = LSTM(60, return_sequences=True)(visible)
dropout1 = Dropout(0.5)(hidden1)
hidden_2 = LSTM(40, return_sequences=False)(dropout1)
dropout_2 = Dropout(0.5)(hidden_2)
out_reg = Dense(1, activation='linear')(dropout_2)
dense_class = Dense(40, activation = 'relu')(dropout_2)
out_class = Dense(1, activation = 'sigmoid')(dense_class)
model = Model(inputs=visible, outputs=[out_reg, out_class])
optimizer = Adam(learning_rate=0.1)
model.compile(loss=['mse',tf.keras.losses.binary_crossentropy], optimizer=optimizer)

In [51]:
model.fit(X_test, [y_train_reg,y_train_cat], epochs=40, batch_size=256, verbose=0)

In [52]:
yhat1, yhat2 = model.predict(X_test)

In [36]:
from sklearn.metrics import accuracy_score, mean_squared_error

In [53]:
accuracy_score(y_test_cat, (yhat2>=0.5)*1)

0.6055776892430279

In [38]:
mean_squared_error(y_train_reg, yhat1) ** (1/2)

0.3909725915080249

In [57]:
X_train_lstm =np.expand_dims(np.array(data_train['stock'].to_list()), axis=1).reshape((1254,20,1))

In [99]:
y_train_cat = data_train['target_classif'].values
y_train_reg = data_train['target_reg']

set()

In [128]:
idx = np.arange(len(X_train_lstm))
np.random.shuffle(idx)
set(train_idx).intersection(set(test_idx))
train_stop = int(0.8 * len(idx))
train_idx = idx[:train_stop]
test_idx = idx[train_stop:]
X_train_nlp = data_lstm[train_idx]
X_test_nlp = data_lstm[test_idx]
X_train_num = X_train_lstm[train_idx]
X_test_num = X_train_lstm[test_idx]
y_train_cat = data_train['target_classif'].values[train_idx]
y_train_reg = data_train['target_reg'].values[train_idx]
y_test_cat = data_train['target_classif'].values[test_idx]
y_test_reg = data_train['target_reg'].values[test_idx]

In [129]:
print(X_train_nlp.shape)
print(X_test_nlp.shape)

(1003, 20, 300)
(251, 20, 300)


In [130]:
input_nlp = Input(shape=(data_lstm.shape[1], data_lstm.shape[2]))
input_num = Input(shape=(X_train_lstm.shape[1], 1))
# the first branch operates on the first input
x = LSTM(100, return_sequences=True)(input_nlp)
x = Dropout(0.5)(x)
x = LSTM(40, return_sequences=False)(x)
x = Dropout(0.5)(x)
x = Model(inputs=input_nlp, outputs=x)
# the second branch opreates on the second input
y = LSTM(100, return_sequences=True)(input_num)
y = Dropout(0.5)(y)
y = LSTM(40, return_sequences=False)(y)
y = Dropout(0.5)(y)
y = Model(inputs=input_num, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])

z = Dense(128, activation='relu')(combined)
out_reg = Dense(1, activation='linear')(z)
# dense_class = Dense(40, activation = 'relu')(dropout_2)
out_class = Dense(1, activation = 'sigmoid')(z)
model = Model(inputs=[x.input, y.input], outputs=[out_reg, out_class])
optimizer = Adam(learning_rate=0.001)
model.compile(loss=['mse',tf.keras.losses.binary_crossentropy], optimizer=optimizer)

In [131]:
model.fit([X_train_nlp, X_train_num], [y_train_reg,y_train_cat], epochs=20, batch_size=256, verbose=0)

In [132]:
yhat1, yhat2 = model.predict([X_test_nlp, X_test_num])

In [133]:
accuracy_score(y_test_cat, (yhat2>=0.5)*1)

0.8446215139442231

In [134]:
mean_squared_error(y_test_reg, yhat1) ** (1/2)

0.2097480263909956

In [83]:
yhat2

array([[1.5328737e-06],
       [1.2821946e-05],
       [9.9999952e-01],
       ...,
       [9.9255274e-05],
       [9.9998736e-01],
       [9.9999797e-01]], dtype=float32)